# Gesture Classification to Palm, Fist, Nothing

### Importing Libraries

In [0]:
import keras
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten, GlobalAveragePooling2D
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam

### Defining model architecture

In [0]:
def build_model():
    base = MobileNet(input_shape=(224,224,3),include_top=False, weights='imagenet')
    classifier = Sequential()
    classifier.add(base)
    classifier.add(GlobalAveragePooling2D())
    classifier.add(Dropout(.6))
    classifier.add(Dense(1024,activation='relu'))
    classifier.add(Dropout(.6))
    classifier.add(Dense(3,activation='sigmoid'))
    for layer in base.layers:
        layer.trainable = False
    for layer in base.layers[-2:]:
        layer.trainable = True
    adam = Adam(.0001)
    classifier.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [32]:
model = build_model()''

#### Image data generator

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [34]:
train_dataloader = datagen.flow_from_directory('../../data/gesture_image/',target_size = (224, 224),batch_size = 256,class_mode = 'categorical')
test_dataloader = datagen.flow_from_directory('../../data/valid_gesture_image/',target_size = (224, 224),batch_size = 256,class_mode = 'categorical')

Found 57138 images belonging to 3 classes.
Found 995 images belonging to 3 classes.


In [35]:
history = model.fit_generator(train_dataloader, steps_per_epoch = 57138//256, epochs = 3, validation_data = test_dataloader)

Epoch 1/3
223/223 [==============================] - 102s 457ms/step - loss: 0.3739 - acc: 0.8440 - val_loss: 0.0656 - val_acc: 0.9849
Epoch 2/3
223/223 [==============================] - 99s 444ms/step - loss: 0.1156 - acc: 0.9594 - val_loss: 0.0495 - val_acc: 0.9859
Epoch 3/3
223/223 [==============================] - 98s 440ms/step - loss: 0.0896 - acc: 0.9686 - val_loss: 0.0441 - val_acc: 0.9899


In [0]:
model.save('gesture_new.h5')